In [ ]:
# %pip install beautifulsoup4
# %pip install requests

In [ ]:
# %pip install urllib3==1.26.7

In [ ]:
import csv
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

In [ ]:
# def generate_page_url(base_url, page_number):
#     page_url = base_url.replace('/page=1/', f'/page={page_number}/')
#     return page_url

def generate_page_url(base_url, page_number):
    url_parts = base_url.split('=')
    url_parts[-1] = str(page_number)
    page_url = '='.join(url_parts)
    return page_url


In [ ]:
# Scrape Link to Detail Page
dashboard_url = 'https://www.lamudi.co.id/west-nusa-tenggara/house/buy/?page=1'
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

for page_number in range(1, 21):  # Ubah 20 menjadi 21 untuk memperoleh halaman 1 hingga 20
    current_page_url = generate_page_url(dashboard_url, page_number)
    driver.get(current_page_url)

    # STATIS
    # soup = BeautifulSoup(driver.page_source, 'html.parser')  # Menggunakan driver.page_source untuk konten halaman
    # links = soup.find_all('a', class_='ListingCell-moreInfo-button-v2_redesign')
    # for link in links:
    #     print(link['href'])

    # Tunggu hingga konten dinamis selesai dimuat
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ListingCell-moreInfo-button-v2_redesign')))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('a', class_='ListingCell-moreInfo-button-v2_redesign')
    
    for link in links:
        print(link['href'])
    
    print(len(links))
    
driver.quit()  # Tutup browser setelah selesai scraping


In [ ]:
len(links)

In [ ]:
driver = webdriver.Chrome()

def generate_page_url(base_url, page_number):
    page_url = base_url.replace('/page.1/', f'/page.{page_number}/')
    return page_url

dashboard_url = 'https://rumah.trovit.co.id/index.php/cod.search_adwords_homes/type.1/what_d.Nusa%20Tenggara%20Barat/origin.1/ppc_landing_type.2/rooms_min.0/bathrooms_min.0/property_type.Rumah/region.Nusa Tenggara Barat/order_by.relevance/resultsPerPage.25/isUserSearch.1/page.1'
page_number = 1

while True:
    current_page_url = generate_page_url(dashboard_url, page_number)
    driver.get(current_page_url)
    # WebDriverWait(driver, 10).until(EC.url_to_be(current_page_url))
    response = requests.get(dashboard_url)

    if response.status_code == 200:
        dashboard_content = response.content
        dashboard_soup = BeautifulSoup(dashboard_content, 'html.parser')


        harga_element = dashboard_soup.find('div', class_='item-price')
        harga = harga_element.find('span', class_='actual-price').text.strip()


        links = dashboard_soup.find_all('a', class_='rd-link image-link')


        data_properti = []

        i = 0

        for link in links:
            i += 1
            detail_url = link['href']
            driver.get(detail_url)
            print("GO! ", i)
            try:
                print("Menemukan (Menampilkan Lebih Banyak)")
                link_element = driver.find_element(By.CLASS_NAME, 'relative ui-content-half__selector')
                driver.execute_script("arguments[0].click();", link_element)
                time.sleep(2)  
                detail_content = driver.page_source

                try: 
                    iklan_content = driver.find_element(By.CLASS_NAME, 'pinder modal')
                    iklan_soup = BeautifulSoup(iklan_content, 'html.parser')
                    close_button = iklan_soup.find('button', class_='button_close')
                    print("ada iklan")
                    if close_button:
                        driver.execute_script("arguments[0].click();", close_button)

                except NoSuchElementException:
                    continue

                try:
                    link_element = driver.find_element(By.CLASS_NAME, 'ui-listing-specification__table--row')
                    # WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ui-listing-specification__table--row')))
                    detail_content = driver.page_source
                    print("Menemukan (Row Spek)")

                except NoSuchElementException:
                    continue

            except NoSuchElementException:
                continue

            detail_soup = BeautifulSoup(detail_content, 'html.parser')
            lokasi_element = detail_soup.find('div', class_='r123-listing-summary__header-container-address')
            lokasi = lokasi_element.text.strip() if lokasi_element else ''
            spesifikasi_element = detail_soup.find('div', class_='ui-listing-specification__table')

            if spesifikasi_element is None:
                continue

            rows = spesifikasi_element.find_all('div', class_='ui-listing-specification__table--row')
            properti = {}

            for row in rows:
                label = row.find('p', class_='ui-listing-specification__table--label').text.strip()
                value = row.find('p', class_='ui-listing-specification__table--value').text.strip()
                properti[label] = value

            properti['Harga'] = harga
            properti['Lokasi'] = lokasi
            data_properti.append(properti)

        header = ['K. Tidur', 'K. Mandi', 'L. Tanah', 'L. Bangunan', 'Tipe Properti', 'Sertifikat', 'Daya Listrik', 'ID Iklan', 'Harga', 'Lokasi']
        with open('data_properti.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=header)
            writer.writeheader()
            writer.writerows(data_properti)

        print('Web scraping selesai. Data properti telah disimpan dalam file data_properti.csv')
    else:
        print('Gagal mengakses halaman dashboard')
    
    try:
        next_button = driver.find_element(By.XPATH, '//p[@class="trovit-paginator"]/a[@class="trovit-button no-background next"]')
        if not next_button.is_enabled():
            break
    except:
        break
    
    page_number += 1

driver.quit()


In [ ]:
# # URL halaman dashboard
# dashboard_url = 'https://rumah.trovit.co.id/index.php/cod.search_adwords_homes/ppc_landing_type.3/type.1/what_d.Nusa%20Tenggara%20Barat/sug.0/isUserSearch.1/origin.1/order_by.relevance/region.Nusa%20Tenggara%20Barat/property_type.Rumah/'
# # Ganti dengan URL halaman dashboardmu

# # Mengirim permintaan HTTP ke halaman dashboard
# response = requests.get(dashboard_url)

# if response.status_code == 200:
#     dashboard_content = response.content
#     dashboard_soup = BeautifulSoup(dashboard_content, 'html.parser')

#     # Mendapatkan harga dari halaman dashboard
#     harga_element = dashboard_soup.find('div', class_='item-price')
#     harga = harga_element.find('span', class_='actual-price').text.strip()

#     # Mendapatkan link detail properti dari halaman dashboard
#     links = dashboard_soup.find_all('a', class_='rd-link image-link')

#     # Inisialisasi list untuk menyimpan data properti
#     data_properti = []

#     # Inisialisasi WebDriver Selenium
#     driver = webdriver.Chrome()  # Ganti dengan driver sesuai preferensi (misal: Firefox, Safari)

#     # Loop melalui setiap link detail properti
#     for link in links:
#         detail_url = link['href']
#         driver.get(detail_url)

#         try:
#             # Cari elemen teks link dan klik
#             # link_element = driver.find_element(By.XPATH, '//div[contains(text(), "Menampilkan lebih banyak")]')
#             # driver.execute_script("arguments[0].click();", link_element)
#             # Cari elemen teks link dan klik

#             link_element = driver.find_element(By.CLASS_NAME, 'relative ui-content-half__selector')
#             driver.execute_script("arguments[0].click();", link_element)

#             # Tunggu beberapa detik
#             time.sleep(2)  # Ubah angka 5 menjadi jumlah detik yang Anda anggap perlu

#             # Ambil konten HTML setelah modal view muncul
#             detail_content = driver.page_source

#             # Ada iklan
#             iklan_content = driver.find_element(By.CLASS_NAME, 'pinder modal')

#             # Setelah Anda mendapatkan konten HTML modal iklan
#             iklan_soup = BeautifulSoup(iklan_content, 'html.parser')

#             # Cari elemen tombol penutup
#             close_button = iklan_soup.find('button', class_='button_close')

#             # Jika tombol penutup ditemukan, klik tombol tersebut
#             if close_button:
#                 driver.execute_script("arguments[0].click();", close_button)


#             # Tunggu hingga modal view muncul
#             try:
#                 WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'ui-listing-specification__table--row')))

#                 # link_detil = driver.find_element(By.CLASS_NAME, 'ui-listing-specification__table--row')
#                 # Ambil konten HTML setelah modal view muncul
#                 detail_content = driver.page_source
#             except NoSuchElementException:
#                 # Skip data properti jika tidak ditemukan elemen "Menampilkan lebih banyak"
#                 continue

#         except NoSuchElementException:
#             # Skip data properti jika tidak ditemukan elemen "Menampilkan lebih banyak"
#             continue



#         # Buat objek BeautifulSoup baru dari konten HTML yang diperoleh
#         detail_soup = BeautifulSoup(detail_content, 'html.parser')

#         # Mendapatkan lokasi dari halaman detail properti
#         lokasi_element = detail_soup.find('div', class_='r123-listing-summary__header-container-address')
#         lokasi = lokasi_element.text.strip() if lokasi_element else ''

#         # Mendapatkan spesifikasi properti dari halaman detail properti
#         # spesifikasi_element = detail_soup.find('div', class_='ui-listing-specification__table')
#         # rows = spesifikasi_element.find_all('div', class_='ui-listing-specification__table--row')
#         # Mendapatkan spesifikasi properti dari halaman detail properti
#         spesifikasi_element = detail_soup.find('div', class_='ui-listing-specification__table')
#         if spesifikasi_element is None:
#             continue

#         rows = spesifikasi_element.find_all('div', class_='ui-listing-specification__table--row')


#         # Membuat dictionary untuk menyimpan data spesifikasi properti
#         properti = {}

#         for row in rows:
#             label = row.find('p', class_='ui-listing-specification__table--label').text.strip()
#             value = row.find('p', class_='ui-listing-specification__table--value').text.strip()
#             properti[label] = value

#         # Menambahkan harga dan lokasi ke dictionary properti
#         properti['Harga'] = harga
#         properti['Lokasi'] = lokasi

#         # Menambahkan dictionary properti ke list data_properti
#         data_properti.append(properti)

#     driver.quit()

#     # Menyimpan data properti ke dalam file CSV
#     header = ['K. Tidur', 'K. Mandi', 'L. Tanah', 'L. Bangunan', 'Tipe Properti', 'Sertifikat', 'Daya Listrik', 'ID Iklan', 'Harga', 'Lokasi']
#     with open('data_properti.csv', 'w', newline='', encoding='utf-8') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=header)
#         writer.writeheader()
#         writer.writerows(data_properti)

#     print('Web scraping selesai. Data properti telah disimpan dalam file data_properti.csv')
# else:
#     print('Gagal mengakses halaman dashboard')


In [ ]:
len(links)

In [ ]:
data_properti